In [22]:
import pandas as pd
import numpy as np

In [23]:
data = pd.read_csv('Data.csv')
data["Date"] = pd.to_datetime(data["Date"])

In [24]:
data.dtypes

Entity                                     object
Continent                                  object
Latitude                                  float64
Longitude                                 float64
Average temperature per year                int64
Hospital beds per 1000 people             float64
Medical doctors per 1000 people           float64
GDP/Capita                                float64
Population                                  int64
Median age                                  int64
Population aged 65 and over (%)             int64
Date                               datetime64[ns]
Daily tests                               float64
Cases                                     float64
Deaths                                    float64
dtype: object

In [25]:
data.isnull().sum()

Entity                                0
Continent                             0
Latitude                              0
Longitude                             0
Average temperature per year          0
Hospital beds per 1000 people         0
Medical doctors per 1000 people       0
GDP/Capita                            0
Population                            0
Median age                            0
Population aged 65 and over (%)       0
Date                                  0
Daily tests                        7895
Cases                               254
Deaths                             3610
dtype: int64

In [26]:

data = data.sort_values(by=['Entity', 'Date']).groupby('Entity').apply(lambda x: x.fillna(method='ffill')) # forward filling null values (i.e. fill null values with the values of the previous row)
data = data.sort_values(by=['Entity', 'Date']).groupby('Entity').apply(lambda x: x.fillna(method='bfill')) # backward filling null values (i.e. fill null values with the values of the following row)

C:\Users\Manika Hennedige\AppData\Local\Temp\ipykernel_35036\1374660882.py:1: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  data = data.sort_values(by=['Entity', 'Date']).groupby('Entity').apply(lambda x: x.fillna(method='ffill')) # forward filling null values (i.e. fill null values with the values of the previous row)
C:\Users\Manika Hennedige\AppData\Local\Temp\ipykernel_35036\1374660882.py:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous beh

In [27]:
len(data["Entity"].unique()) # 104 unique countries

104

In [28]:
data["TestColumn"]  = data["Entity"]
common_cols = [] # columns that are (based on the data) dependent on the 'Entity' column
for colname in data.columns: 
    if colname != "Entity" and colname != "TestColumn":
        data["NewTestColumn"] = data["TestColumn"] + " " + data[colname].astype(str)
        if len(data["NewTestColumn"].unique()) == len(data["Entity"].unique()):
            common_cols.append(colname)
            data["TestColumn"] = data["NewTestColumn"]
        else:
            print(colname + " is not dependent on Entity")
            

Date is not dependent on Entity
Daily tests is not dependent on Entity
Cases is not dependent on Entity
Deaths is not dependent on Entity


In [29]:
common_cols # these are the columns that appear to be dependent on the Entity

['Continent',
 'Latitude',
 'Longitude',
 'Average temperature per year',
 'Hospital beds per 1000 people',
 'Medical doctors per 1000 people',
 'GDP/Capita',
 'Population',
 'Median age',
 'Population aged 65 and over (%)']

### Proposed Database Schema

#### This database schema is based on normalizing the data as much as possible to 3NF
##### Note that realistically, with a dataset spanning a longer period of time, a different schema may need to be used.
For example, if 'average temperature per year' was observed to have different values for a specific entity, an additional 'Year' column may need to be generated and associated with 'average temperature per year'. In this case, since each entity only has one unique value, I have opted for a simplified schema suitable for this data.

1. Entity
    - EntityId
    - EntityName
    - Latitude
    - Longitude
    - Average temperature per year
    - Hospital beds per 1000 people
    - Medical doctors per 1000 people
    - GDP/Capita
    - Population
    - Median Age
    - Population aged 65 and over (%)
2. Continent
    - ContinentId
    - ContinentName
3. ContinentEntity
    - ContinentEntityId
    - ContinentId
    - EntityId
4. EntityDate
    - EntityDateId
    - EntityId
    - Date
    - DailyTests
    - Cases
    - Deaths